# DSS Performance Optimizations

Thomas Chamberlin, Green Bank Observatory

In [1]:
import time
import os

os.environ["DJANGO_SETTINGS_MODULE"] = "nell.conf.development"

import django
from django.db import connection
django.setup()

from scheduler.models import Project, Period, User

# Disable errors about SynchronousOnlyOperation
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sqlparse

def query_profile(func):
    def wrapper(*args, **kwargs):
        num_initial_queries = len(connection.queries)
        start_time = time.perf_counter()
        ret = func(*args, **kwargs)
        end_time = time.perf_counter()
        num_final_queries = len(connection.queries)
        num_queries = num_final_queries - num_initial_queries
        total_query_time = sum(float(q["time"]) for q in connection.queries[num_initial_queries:num_final_queries])
        print(f"{func.__name__}: {num_queries} quer{'y' if num_queries == 1 else 'ies'} in {total_query_time:.3f} seconds; {end_time - start_time:.4f} seconds total")
        return ret
    return wrapper

def query_profile_verbose(func):
    def wrapper(*args, **kwargs):
        num_initial_queries = len(connection.queries)
        start_time = time.perf_counter()
        ret = func(*args, **kwargs)
        end_time = time.perf_counter()
        num_final_queries = len(connection.queries)
        num_queries = num_final_queries - num_initial_queries
        total_query_time = sum(float(q["time"]) for q in connection.queries[num_initial_queries:num_final_queries])
        print(f"{func.__name__}: {num_queries} quer{'y' if num_queries == 1 else 'ies'} in {total_query_time:.3f} seconds; {end_time - start_time:.4f} seconds total")
        for query in connection.queries[num_initial_queries:num_final_queries]:
            print(sqlparse.format(query["sql"], reindent=True, keyword_case='upper'))
        return ret
    return wrapper

## The DSS `scheduler` Models/ERD

![DSS Scheduler Models ERD](./scheduler_erd.svg)

## Get Periods for Project

A `Project` has logic to determine all of its `Period`s. This requires `O(n)` queries.

Reminder: A `Project` has `Session`s, and a `Session` has `Period`s

In [15]:
# scheduler.Project.getPeriods
@query_profile
def getPeriods(project):
    """What are the periods associated with this project, visible to observer?"""
    return sorted(
        [
            p
            # Pull EVERY Session object out of the DB (1 query)
            for s in project.sesshun_set.all()
            # The actual queries. (s.sesshun_set.count() queries)
            for p in s.period_set.exclude(state__abbreviation="P").exclude(
                state__abbreviation="D"
            )
        ],
        # Sort list in Python, rather than in DB
        key=lambda proposal: proposal.start
    )

project = Project.objects.get(pcode="GBT21B-996")
periods = getPeriods(project)
print(f"{len(periods)=}")

getPeriods: 79 queries in 0.071 seconds; 0.1901 seconds total
len(periods)=50


This can easily be fixed with proper use of filters across relationships

In [3]:
# scheduler.Project.getPeriods
@query_profile_verbose
def getPeriods__fixed(project):
    """What are the periods associated with this project, vis. to observer?"""
    # Convert to  list to force QS evaluation (this isn't in the real code)
    return list(
        Period.objects.filter(session__project=project)
        .exclude(state__abbreviation__in=["P", "D"])
        .order_by("start")
    )

project = Project.objects.get(pcode="GBT21B-996")
periods = getPeriods__fixed(project)

getPeriods__fixed: 1 query in 0.003 seconds; 0.0081 seconds total
SELECT "periods"."id",
       "periods"."session_id",
       "periods"."accounting_id",
       "periods"."state_id",
       "periods"."start",
       "periods"."duration",
       "periods"."score",
       "periods"."forecast",
       "periods"."backup",
       "periods"."moc",
       "periods"."moc_ack",
       "periods"."window_id",
       "periods"."elective_id",
       "periods"."last_notification"
FROM "periods"
INNER JOIN "sessions" ON ("periods"."session_id" = "sessions"."id")
LEFT OUTER JOIN "period_states" ON ("periods"."state_id" = "period_states"."id")
WHERE ("sessions"."project_id" = 2336
       AND NOT ("period_states"."abbreviation" IN ('P',
                                                   'D')
                AND "period_states"."abbreviation" IS NOT NULL))
ORDER BY "periods"."start" ASC


## Is a User an Investigator for a given Project?

Reminder: a `User` is an `Investigator` for some number of `Project`s

In [4]:
# scheduler.User.isInvestigator
@query_profile
def isInvestigator(user, pcode):
    """Is this user an investigator on the given project?"""
    # Create a list of the pcodes for all Projects on which this User is an Investigator
    return pcode in [i.project.pcode for i in user.investigator_set.all()]

tm = User.objects.get(first_name="Toney", last_name="Minter")
print(f"{tm} is linked to {tm.investigator_set.count()} Investigators")
isInvestigator(tm, "GBT21B-996")

Minter, Toney is linked to 73 Investigators
isInvestigator: 74 queries in 0.064 seconds; 0.1137 seconds total


False

In [5]:
# scheduler.User.isInvestigator
@query_profile_verbose
def isInvestigator_fixed(user, pcode):
    """Is this user an investigator on the given project?"""
    return user.investigator_set.filter(project__pcode=pcode).exists()

tm = User.objects.get(first_name="Toney", last_name="Minter")
isInvestigator_fixed(tm, "GBT21B-996")

isInvestigator_fixed: 1 query in 0.001 seconds; 0.0021 seconds total
SELECT (1) AS "a"
FROM "investigators"
INNER JOIN "projects" ON ("investigators"."project_id" = "projects"."id")
WHERE ("investigators"."user_id" = 7
       AND "projects"."pcode" = 'GBT21B-996')
LIMIT 1


False

## Get Authors

We're in the PHT now (moving away from the scheduler).

A `Proposal` has some number of `Author`s

There is a View in the PHT that simply serializes all `Author`s. In doing so, it generates N+1 queries.

In [6]:
from pht.models import Author
# Adapted from pht.views.pis
@query_profile
def get_authors():
    authors = [
        {"id": a.id, "name": a.getLastFirstName(), "pcode": a.proposal.pcode}
        # Artifically limit to 100 Authors simply to keep total number of queries down
        for a in Author.objects.all()[:100]
    ]
    return authors

authors = get_authors()
print(f"{len(authors)=}")

Author.objects.values("id", "name", "pcode")

get_authors: 101 queries in 0.246 seconds; 0.3872 seconds total
len(authors)=100


Django provides an easy way to get a "list of dicts" version of a `QuerySet`: `QuerySet.values()`. But we have a method call to `Author.getLastFirstName`, which complicates things. What is that doing?

```python
def getLastFirstName(self):
    return "%s, %s" % (self.last_name, self.first_name)
```

Great, it's literally just concatenating the last and first names together (with separator characters). We can do this in the database via Django's `Concat` DB function.

In [16]:
from pht.models import Author
from django.db.models import F, Value
from django.db.models.functions import Concat

# Adapted from pht.views.pis
@query_profile_verbose
def get_authors():
    authors = list(
        Author.objects.annotate(
            # Create a new column named pcode so the results dicts have the correct keys
            pcode=F("proposal__pcode"),
            # Concatenate last name and first name together, with the required separators
            name=Concat(F("last_name"), Value(", "), F("first_name")),
        ).values("id", "name", "pcode")[:100]
    )
    return authors

authors = get_authors()
print(f"{len(authors)=}")

get_authors: 1 query in 0.002 seconds; 0.0042 seconds total
SELECT "pht_authors"."id",
       "pht_proposals"."pcode" AS "pcode",
       CONCAT("pht_authors"."last_name", CONCAT(', ', "pht_authors"."first_name")) AS "name"
FROM "pht_authors"
INNER JOIN "pht_proposals" ON ("pht_authors"."proposal_id" = "pht_proposals"."id")
LIMIT 100
len(authors)=100


## Get `Period`s in Time Range

`scheduler.models.Period.in_time_range` is a method that returns the `Period`s in a given time range.
```

In [8]:
from datetime import datetime, timedelta
import pytz
from scheduler.models import Period

# Adapted from: scheduler.models.Period.in_time_range
@query_profile
def in_time_range(begin, end, ignore_deleted=True):
    """
    Returns all periods in a time range, taking into account that periods
    can overlap into the first day.
    """
    ps = Period.objects.filter(start__gt=begin - timedelta(days=1), start__lt=end).order_by("start")
    ps = [p for p in ps if p.end() > begin]

    if ignore_deleted:
        ps = [p for p in ps if p.state.abbreviation != "D"]

    return ps

periods = in_time_range(begin=datetime(2018, 12, 1), end=datetime(2019, 1, 1))
print(f"{len(periods)=}")

in_time_range: 268 queries in 0.062 seconds; 0.3139 seconds total
len(periods)=244


Not great! We are making _way_ too many queries. It might seem like another simple solution: simply filter against the `end` field, like we're doing for `start`. But `end` _isn't a field_, it's a method! For reference, here are the relevant parts of the `Period` model:

```python
class Period(models.Model):
    session = models.ForeignKey("Session", on_delete=models.CASCADE)
    start = models.DateTimeField(help_text="yyyy-mm-dd hh:mm")
    duration = models.FloatField(help_text="Hours")

    def end(self):
        """The period ends at start + duration."""
        return self.start + timedelta(hours=self.duration)
```

To work around this problem, we can _derive_ an `end` column at query-time using QuerySet annotations. However, the fact that `Period.duration` is a discrete number of hours, and not a proper "duration" field, complicates things.

We end up using the `interval` function in PostgreSQL to convert the discrete hours to an interval type, and put this value into a new `end` column.

In [9]:
from datetime import datetime, timedelta
import pytz
from django.db import models
from scheduler.models import Period

# Our custom DB Function
class Hours(models.Func):
    """Convert a "number of hours" column into an proper interval type"""

    function = "interval"
    template = "%(expressions)s * %(function)s '1 hour'"

# Adapted from: scheduler.models.Period.in_time_range
@query_profile_verbose
def in_time_range__fixed(begin, end, ignore_deleted=True):
    """
    Returns all periods in a time range, taking into account that periods
    can overlap into the first day.
    """
    ps = (
        Period.objects.annotate(
            end=models.F("start")
            + Hours(models.F("duration"), output_field=models.DateTimeField())
        )
        .filter(start__gt=begin - timedelta(days=1), start__lt=end)
        .filter(end__gt=begin)
        .order_by("start")
    )

    if ignore_deleted:
        ps = ps.exclude(state__abbreviation="D")

    return list(ps)

periods = in_time_range__fixed(begin=datetime(2018, 12, 1), end=datetime(2019, 1, 1))
print(f"{len(periods)=}")

in_time_range__fixed: 1 query in 0.012 seconds; 0.0230 seconds total
SELECT "periods"."id",
       "periods"."session_id",
       "periods"."accounting_id",
       "periods"."state_id",
       "periods"."start",
       "periods"."duration",
       "periods"."score",
       "periods"."forecast",
       "periods"."backup",
       "periods"."moc",
       "periods"."moc_ack",
       "periods"."window_id",
       "periods"."elective_id",
       "periods"."last_notification",
       ("periods"."start" + "periods"."duration" * interval '1 hour') AS "end"
FROM "periods"
LEFT OUTER JOIN "period_states" ON ("periods"."state_id" = "period_states"."id")
WHERE ("periods"."start" > '2018-11-30T00:00:00'::TIMESTAMP
       AND "periods"."start" < '2019-01-01T00:00:00'::TIMESTAMP
       AND ("periods"."start" + "periods"."duration" * interval '1 hour') > '2018-12-01T00:00:00'::TIMESTAMP
       AND NOT ("period_states"."abbreviation" = 'D'
                AND "period_states"."abbreviation" IS NOT NULL))

## Get Unique Telescopes

The OpsLog Shift Summary tool includes code that derives all of the unique `Telescope`s from a given set of `Entry` objects. This ends up being `2N+1` queries

In [10]:
from ops.models import Entry, Telescope

# Adapted from ops.tools.SummaryHtml.ShiftSummaryHtml._makeLostTimeSection
@query_profile
def get_unique_telescopes_from_entries(entries):
    telescopes = set(
        [
            e.telescope
            for e in entries
            if (e is not Telescope.objects.get(name="ALL"))
            and (e.telescope is not None)
        ]
    )
    return telescopes

entries = Entry.objects.all()[:100]
telescopes = get_unique_telescopes_from_entries(entries)
print(f"{telescopes=}")

get_unique_telescopes_from_entries: 201 queries in 0.061 seconds; 0.2278 seconds total
telescopes={<Telescope: GBT>, <Telescope: ALL>}


This can easily be fixed by using `QuerySet.distinct()` and some nested `QuerySet`s

In [11]:
from ops.models import Entry, Telescope

# Adapted from ops.tools.SummaryHtml.ShiftSummaryHtml._makeLostTimeSection
@query_profile_verbose
def get_unique_telescopes_from_entries__fixed(entries):
    telescopes = Telescope.objects.filter(entry__in=entries.exclude(telescope__name="ALL")).distinct()
    return list(telescopes)

entries = Entry.objects.filter(id__in=Entry.objects.all()[:1000])
telescopes = get_unique_telescopes_from_entries__fixed(entries)
print(f"{telescopes=}")

get_unique_telescopes_from_entries__fixed: 1 query in 0.062 seconds; 0.0650 seconds total
SELECT DISTINCT "ops_telescope"."id",
                "ops_telescope"."name"
FROM "ops_telescope"
INNER JOIN "ops_entry" ON ("ops_telescope"."id" = "ops_entry"."telescope_id")
WHERE "ops_entry"."id" IN
    (SELECT V0."id"
     FROM "ops_entry" V0
     LEFT OUTER JOIN "ops_telescope" V1 ON (V0."telescope_id" = V1."id")
     WHERE (V0."id" IN
              (SELECT "ops_entry"."id"
               FROM "ops_entry"
               LIMIT 1000)
            AND NOT (V1."name" = 'ALL'
                     AND V1."name" IS NOT NULL)))
telescopes=[<Telescope: 140'>, <Telescope: GBT>]


## Convert Periods to iCal Events

The `IcalMap` contains logic for converting `scheduler` `Period`s into `icalendar` `Event`s. Unfortunately, it was taking `4n+1` (`O(n)`) queries to do so.

In [12]:
from datetime import datetime, timedelta
from pytz import UTC
from icalendar import Calendar, Event
from scheduler.models import User

def createPeriodEvent(period):
        event = Event()
        event["uid"] = str(period.id) + "periodofgbtdss"
        start = datetime(
            period.start.year,
            period.start.month,
            period.start.day,
            period.start.hour,
            period.start.minute,
            period.start.second,
            tzinfo=UTC,
        )
        event.add("dtstart", start)
        event.add("dtend", start + timedelta(hours=period.duration))
        event.add("dtstamp", datetime.utcnow())
        name = (
            period.session.project.pcode in period.session.name
            and period.session.name
            or (period.session.name + " of " + period.session.project.pcode)
        )
        event.add(
            "summary",
            "%s at %.3f GHz (%s UTC)"
            % (name, period.session.frequency, start.strftime("%Y/%m/%d %H:%M")),
        )
        event.add(
            "description",
            "%s. This telescope period has a duration of %.2f hours. The receiver requirements for this telescope period are %s. The cover page containing all project details is at http://gbrescal.gb.nrao.edu/gbtobs/proposals.dbw?view=viewproposal&propcode=%s"
            % (
                period.session.project.name,
                period.duration,
                period.session.receiver_list(),
                period.session.project.pcode,
            ),
        )
        event.add("priority", 9)

        return event

# Adapted from tools.IcalMap.IcalMap.__init__
def get_future_scheduled_periods(user):
    periods = (
        user.getPeriods()
        .filter(state__abbreviation="S", start__gte=datetime.now())
        .order_by("start")
    )
    return periods

@query_profile
def convert_periods_to_events(user):
    periods = get_future_scheduled_periods(user)
    events = [createPeriodEvent(period) for period in periods]
    return events

tm = User.objects.get(first_name="Toney", last_name="Minter")
events = convert_periods_to_events(tm)
print(f"{len(events)=}")

convert_periods_to_events: 197 queries in 0.130 seconds; 0.3449 seconds total
len(events)=49


In [14]:
@query_profile_verbose
def convert_periods_to_events(user):
    periods = get_future_scheduled_periods(user)
    periods = periods.select_related("session__project").prefetch_related("session__receiver_group_set__receivers")
    events = [createPeriodEvent(period) for period in periods]
    return events

tm = User.objects.get(first_name="Toney", last_name="Minter")
events = convert_periods_to_events(tm)
print(f"{len(events)=}")

convert_periods_to_events: 3 queries in 0.013 seconds; 0.0427 seconds total
SELECT "periods"."id",
       "periods"."session_id",
       "periods"."accounting_id",
       "periods"."state_id",
       "periods"."start",
       "periods"."duration",
       "periods"."score",
       "periods"."forecast",
       "periods"."backup",
       "periods"."moc",
       "periods"."moc_ack",
       "periods"."window_id",
       "periods"."elective_id",
       "periods"."last_notification",
       "sessions"."id",
       "sessions"."project_id",
       "sessions"."session_type_id",
       "sessions"."observing_type_id",
       "sessions"."allotment_id",
       "sessions"."status_id",
       "sessions"."original_id",
       "sessions"."name",
       "sessions"."frequency",
       "sessions"."max_duration",
       "sessions"."min_duration",
       "sessions"."time_between",
       "sessions"."accounting_notes",
       "sessions"."notes",
       "projects"."id",
       "projects"."semester_id",
       